In [29]:
!pip install pinecone-client[grpc] sentence-transformers octoai openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.5/327.5 kB 4.7 MB/s eta 0:00:00


In [15]:
from pinecone.grpc import PineconeGRPC as Pinecone
from sentence_transformers import SentenceTransformer

pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY")
index = pc.Index("hackathon")

def create_embedding(query):
  model = SentenceTransformer('BAAI/bge-base-en-v1.5')
  return model.encode(query, normalize_embeddings=True)

def query_pinecone(namespace, query, top_k=10):
  embedding = create_embedding(query)
  resp = index.query(
    namespace=namespace,
    vector=embedding,
    top_k=top_k,
    include_values=True,
    include_metadata=True
  )
  return resp


In [16]:
print(query_pinecone('codebase', ''))

{'matches': [{'id': 'b5760996-7167-43e7-aaea-9a93c2b4ab19',
              'metadata': {'chunk_num': 0.0,
                           'issue_num': '6896',
                           'text': 'Issue Number: 6896\n'
                                   'Issue URL: '
                                   'https://github.com/python/mypy/pull/6896\n'
                                   'Issue Title: Use Iterator in documentation '
                                   'examples for generator functions\n'
                                   'Issue Labels: \n'
                                   'Issue Body: The current "cheat sheet" '
                                   'examples use `Iterable` to annotate the '
                                   'return type of generator functions.  This '
                                   'PR changes this to `Iterator`, which is '
                                   'more specific.\n'
                                   '\n'
                                   "In most cas

In [23]:
import json

from octoai.client import OctoAI
from octoai.text_gen import ChatMessage
import os

client = OctoAI(api_key=os.getenv("OCTOAI_API_KEY"))

def ask_llm(client_issue, code_matches, issues_matches):
  completion = client.text_gen.create_chat_completion(
      model="meta-llama-3-70b-instruct",
      messages=[
          ChatMessage(
              role="system",
              content=f'You are a coding assistant. You will be given context of relevant code from a user to understand the coding style and issues it corresponds to. Your job is to write fully functional working code for the given issue in hand as described by the user. Here is the code matches: ${code_matches} and here are the issue_matches: ${issues_matches}',
          ),
          ChatMessage(role="user", content=f"Here is the issue that the user wants you to write the code for ${client_issue}"),
      ],
  )

  resp = json.dumps(completion.dict(), indent=2)
  print(resp)
  return resp



In [30]:
from openai import OpenAI
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

def ask_llm_openai(client_issue, code_matches, issues_matches):
  completion = client.chat.completions.create(
    model="gpt-4o",
    messages=[
      {"role": "system", "content": f"You are a coding assistant. You will be given context of relevant code from a user to understand the coding style and issues it corresponds to. Your job is to write fully functional working code for the given issue in hand as described by the user. Here is the code matches: ${code_matches} and here are similar issues and in the metadata there is code on how these issues were solved: ${issues_matches}"},
      {"role": "user", "content": f"Here is the issue that the user wants you to write the code for ${client_issue}"}
    ]
  )

  resp = completion.choices[0].message
  print(resp)
  return resp

In [38]:

def get_pr_from_issue(issue_text):
  query_code_matches = query_pinecone('codebase', issue_text, 2)
  print(query_code_matches)
  query_issue_matches = query_pinecone('issues_and_prs_prod', issue_text, 2)
  print(query_issue_matches)
  resp = ask_llm_openai(issue_text, query_code_matches, [])
  return resp


In [36]:
from IPython.display import display, Markdown

issue = """
Crash Report

Compiled mypy has a segmentation fault.

Traceback

(Too many lines to copy)

To Reproduce

import typing

T1 = typing.NewType('T1', str)
T2 = tuple[T1, 'T4', 'T4']
T3 = tuple[str, 'T4', 'T4']
T4 = T2 | T3

Your Environment

Mypy version used: master (mypy 1.11.0+dev.35fbd2a852be2c47e8006429afe9b3ad4b1bfac2 (compiled: no))
Mypy command-line flags: mypy test.py --show-traceback
Mypy configuration options from mypy.ini (and other config files): No config files
Python version used: 3.12 (compiled from source)
Operating system and version: CentOS 7
"""

generated_code = get_pr_from_issue(issue)
print(generated_code)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


ChatCompletionMessage(content="The issue you're encountering appears to be related to creating a `typing.NewType` and type aliasing with forward references that are causing mypy to crash with a segmentation fault.\n\nLet's try to reproduce and provide the correct version that should prevent the segmentation fault. Here is the corrected version of the code:\n\n```python\nimport typing\n\nT1 = typing.NewType('T1', str)\nT4 = typing.TypeVar('T4')\nT2 = tuple[T1, T4, T4]\nT3 = tuple[str, T4, T4]\nCombinedType = T2 | T3\n\ndef dummy_function(data: CombinedType) -> None:\n    print(data)\n\ndummy_function((T1('x'), 'y', 'z'))\ndummy_function(('a', 'b', 'c'))\n```\n\nExplanation:\n1. Created a `TypeVar` named `T4` to use it as a generic type.\n2. Defined tuple types `T2` and `T3` with `TypeVar` to prevent the use of forward references.\n3. Combined `T2` and `T3` into `CombinedType`.\n4. Added a dummy function to utilize `CombinedType` and demonstrate functionality.\n\nWith the corrected struc

In [37]:
from IPython.display import display, Markdown
display(Markdown(generated_code.content))

The issue you're encountering appears to be related to creating a `typing.NewType` and type aliasing with forward references that are causing mypy to crash with a segmentation fault.

Let's try to reproduce and provide the correct version that should prevent the segmentation fault. Here is the corrected version of the code:

```python
import typing

T1 = typing.NewType('T1', str)
T4 = typing.TypeVar('T4')
T2 = tuple[T1, T4, T4]
T3 = tuple[str, T4, T4]
CombinedType = T2 | T3

def dummy_function(data: CombinedType) -> None:
    print(data)

dummy_function((T1('x'), 'y', 'z'))
dummy_function(('a', 'b', 'c'))
```

Explanation:
1. Created a `TypeVar` named `T4` to use it as a generic type.
2. Defined tuple types `T2` and `T3` with `TypeVar` to prevent the use of forward references.
3. Combined `T2` and `T3` into `CombinedType`.
4. Added a dummy function to utilize `CombinedType` and demonstrate functionality.

With the corrected structure, `mypy` should be able to check the types correctly without running into a segmentation fault. 

Make sure to test the code with `mypy` on your environment to verify that the segmentation fault does not occur with this structure.